In [1]:
import numpy as np
from random_data_generator import *
from sequential_estimator import *
import math
from numpy.linalg import inv

In [2]:
def gaussian(mu, var, x):
    sigma = math.sqrt(var)

    a = sigma*math.sqrt(2*math.pi)
    b = -(1/2)*(((x-mu)/sigma)**2)

    return (1/a)*math.exp(b)

In [3]:
def make_design_matrix(x,n):
    A = np.ones((1,n))

    count = 0
    for col in range(n-1,-1,-1):
        A[0,col] = x ** count
        count += 1
    
    return A

In [4]:
def get_newmu_new_var(b,n,a, new_data, first_round: bool, last_mu, last_var):        
    design_matrix = make_design_matrix(new_data[0],n)
    a = 1/a

    if first_round:
        new_mu = inv(np.dot(design_matrix.T,design_matrix)+np.identity(n)*(b/a))
        new_mu = np.dot(new_mu,design_matrix.T)
        new_mu = new_mu*new_data[1]

        new_var = inv(np.dot(a*design_matrix.T,design_matrix)+b*np.identity(n))
        #  posterior_covariance = np.linalg.inv(inv_variance * design.T.dot(design) + precision * np.identity(basis))
    else:
        new_var = inv(last_var) + np.dot(design_matrix.T,design_matrix)*a
        new_var = inv(new_var)

        new_mu = np.dot(inv(last_var),last_mu) + design_matrix.T*new_data[1]*a
        new_mu = np.dot(new_var, new_mu)

    return [new_mu, new_var]



In [5]:
def bayssian_linear(b,n,a,w: list):
    last_mu = 0
    last_var = 0

    count = 0
    while True:
        new_data = linear_model_data_generator(n,a,w)
        if count==0:
            [new_mu,new_var] = get_newmu_new_var(1,4,1,new_data,True,None,None)
        else:
            [new_mu,new_var] = get_newmu_new_var(1,4,1,new_data,False,last_mu, last_var)
        
        if np.abs(last_mu-new_mu).sum() < 0.000001:
            break

        last_mu = new_mu
        last_var = new_var

        count += 1

        


    return last_mu, last_var

In [6]:
mu, var = bayssian_linear(1,4,1,[1,2,3,4])

In [7]:
mu

array([[3.89407273],
       [2.85992347],
       [2.04018483],
       [1.02100495]])